In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import os, time
from tqdm.notebook import tqdm

from ssd_model import SSD300, SSD512
from ssd_utils import PriorUtil
from ssd_data import InputGenerator
from ssd_training import SSDLoss, SSDFocalLoss

from utils.model import load_weights
from utils.training import MetricUtility

#### MS COCO

In [ ]:
from data_coco import GTUtility
gt_util_train = GTUtility('data/cars/train/')
# gt_util_train, gt_util_val = gt_util.split(0.8)
gt_util_val = GTUtility('data/cars/valid/')
experiment = 'ssd512_coco'
#print(gt_util)

### Model

In [ ]:
#model = SSD300(num_classes=gt_util_train.num_classes)
model = SSD512(num_classes=gt_util_train.num_classes)

prior_util = PriorUtil(model)

### Weights

In [ ]:
initial_epoch = 0

#!wget -O ./models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5 https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
weights_path = './models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
layer_list = [('block1_conv1', 'conv1_1'),
              ('block1_conv2', 'conv1_2'),
              ('block2_conv1', 'conv2_1'),
              ('block2_conv2', 'conv2_2'),
              ('block3_conv1', 'conv3_1'),
              ('block3_conv2', 'conv3_2'),
              ('block3_conv3', 'conv3_3'),
              ('block4_conv1', 'conv4_1'),
              ('block4_conv2', 'conv4_2'),
              ('block4_conv3', 'conv4_3'),
              ('block5_conv1', 'conv5_1'),
              ('block5_conv2', 'conv5_2'),
              ('block5_conv3', 'conv5_3')]
#load_weights(model, weights_path, layer_list)

# weights_path = './models/ssd512_voc_weights_fixed.hdf5'
#weights_path = './checkpoints/201709251316_ssd512_coco/weights.040.h5'; initial_epoch = 41
#weights_path = './checkpoints/201709081909_ssd512_voc/weights.034.h5'; initial_epoch = 35
#model.load_weights(weights_path, by_name=True)
load_weights(model, weights_path)

freeze = ['conv1_1', 'conv1_2',
          'conv2_1', 'conv2_2',
          'conv3_1', 'conv3_2', 'conv3_3',
          #'conv4_1', 'conv4_2', 'conv4_3',
          #'conv5_1', 'conv5_2', 'conv5_3',
         ]

### Training

In [ ]:
batch_size = 1

#optimizer = tf.optimizers.SGD(learning_rate=1e-3, momentum=0.9, decay=0, nesterov=True)
optimizer = tf.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

#loss = SSDLoss(alpha=1.0, neg_pos_ratio=3.0)
loss = SSDFocalLoss()

#regularizer = None
regularizer = keras.regularizers.l2(5e-4) # None if disabled

gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size, augmentation=True)
gen_val = InputGenerator(gt_util_val, prior_util, batch_size, model.image_size, augmentation=True)

dataset_train, dataset_val = gen_train.get_dataset(), gen_val.get_dataset()
# debug
# gen_train.get_sample(0)
# gen_val.get_sample(0)

In [ ]:
epochs = 1

iterator_train, iterator_val = iter(dataset_train), iter(dataset_val)

checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment

if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

print(checkdir)

for l in model.layers:
    l.trainable = not l.name in freeze
    if regularizer and l.__class__.__name__.startswith('Conv'):
        model.add_loss(lambda l=l: regularizer(l.kernel))

metric_util = MetricUtility(loss.metric_names, logdir=checkdir)

@tf.function
def step(x, y_true, training=False):
    if training:
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            metric_values = loss.compute(y_true, y_pred)
            total_loss = metric_values['loss']
            if len(model.losses):
                total_loss += tf.add_n(model.losses)
        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    else:
        y_pred = model(x, training=True)
        metric_values = loss.compute(y_true, y_pred)
    return metric_values

for k in tqdm(range(initial_epoch, epochs), 'total', leave=False):
    print('\nepoch %i/%i' % (k+1, epochs))
    metric_util.on_epoch_begin()

    for i in tqdm(range(gen_train.num_batches), 'training', leave=False):
        x, y_true = next(iterator_train)
        metric_values = step(x, y_true, training=True)
        metric_util.update(metric_values, training=True)
    
    model.save_weights(checkdir+'/weights.%03i.h5' % (k+1,))

    for i in tqdm(range(gen_val.num_batches), 'validation', leave=False):
        x, y_true = next(iterator_val)
        metric_values = step(x, y_true, training=False)
        metric_util.update(metric_values, training=False)

    metric_util.on_epoch_end(verbose=1)